In [8]:
import pandas as pd
from matplotlib import pyplot as plt
import json

from ccb.experiment import parse_results
from ccb.experiment.retrieve_runs import retrieve_runs, find_missing_runs


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## observations
* val loss and val accuracy don't start at the same step. Why?
* val accuracy can decrease over time (it doesn't always stay flat). Maybe it's a better indicator of the best hyper-parameter and the best time step.
* Val accuracy is particularly noisy
* many curves overfits right from the beginning and so2sat almost exclusively overfits
* conv4 is often not converged


In [26]:
# directory where all the sweeps experiments are logged
classification_dir =  '/mnt/data/experiments/nils/last_classification_sweeps'
# classification_dir = '/mnt/data/experiments/nils/final_classification_seeded_runs'
segmentation_dir =  '/mnt/data/experiments/nils/last_segmentation_sweeps'
# segmentation_dir = "/mnt/data/experiments/nils/final_segmentation_seeded_runs"

df = retrieve_runs(segmentation_dir, use_cached_csv=False, exp_type="sweep")

df_1x = df[(df["partition_name"] == "1.00x_train") | (df["partition_name"] == "default")]

100%|██████████| 2681/2681 [06:30<00:00,  6.87it/s]


In [32]:
count_df = df[df["partition_name"]=="0.20x_train"].groupby(["model", "dataset", "partition_name"]).size().reset_index()
count_df

,model,dataset,partition_name,0
0,resnet101_DeepLabV3,NeonTree_segmentation,0.20x_train,12
1,resnet101_DeepLabV3,cvpr_chesapeake_landcover,0.20x_train,12
2,resnet101_DeepLabV3,nz_cattle_segmentation,0.20x_train,12
3,resnet101_DeepLabV3,pv4ger_segmentation,0.20x_train,12
4,resnet101_DeepLabV3,smallholder_cashew,0.20x_train,12
5,resnet101_DeepLabV3,southAfricaCropType,0.20x_train,12
6,resnet101_Unet,NeonTree_segmentation,0.20x_train,12
7,resnet101_Unet,cvpr_chesapeake_landcover,0.20x_train,9
8,resnet101_Unet,nz_cattle_segmentation,0.20x_train,12
9,resnet101_Unet,pv4ger_segmentation,0.20x_train,12


In [28]:
# generate best hparam config from sweeps for seeded runs
best_runs = parse_results.find_best_hparam_for_seeds(df)
# print(best_runs)
with open("/mnt/home/climate-change-benchmark/ccb/configs/best_hparams_segmentation.json", "w") as f:
    json.dump(best_runs, f)

In [17]:
type(best_runs)

collections.defaultdict

In [ ]:

df_1x = df[(df["partition_name"] == "1.00x_train") | (df["partition_name"] == "default")]

# df_05x = df[df["partition_name"]=="0.01x_train"]
best_points_df = parse_results.plot_all_models_datasets(df_1x, plot_fn=parse_results.make_plot_sweep(filt_size=10, top_k=6, legend=False), fig_size=(40,40))
plt.tight_layout()


In [ ]:

# plt.rcParams['figure.figsize'] = [40, 40]

parse_results.plot_all_datasets(df_1x, model="ssl_moco_resnet50", plot_fn=parse_results.make_plot_sweep(filt_size=20, top_k=6, legend=True), fig_size=(10,20))
# parse_results.plot_all_datasets(df_1x, model="resnet50")

plt.tight_layout()


In [ ]:
parse_results.plot_all_models_datasets(df_1x, plot_fn=parse_results.make_plot_sweep(filt_size=20, top_k=6, legend=False), fig_size=(40,40))
plt.tight_layout()

In [ ]:
count_df[count_df["model"].str.contains("ssl_moco")]

In [ ]:
d = find_missing_runs(df, 10, "segmentation")

In [ ]:
import json
print(json.dumps(d, indent = 4))

In [ ]:
from pathlib import Path
df.to_csv(Path(classification_dir) / "cached_results.csv")

In [ ]:
import numpy as np
trace = pd.Series(np.linspace(0,1,1000))
print(trace.mean())
print(trace.rolling(5).sum().mean())
print(trace.rolling(5).mean().mean())
print(trace.rolling(5, win_type="gaussian").mean(std=0.1).mean())
print(trace.rolling(5, win_type="gaussian").mean(std=0.5).mean())
print(trace.rolling(5, win_type="gaussian").mean(std=10).mean())

